In [ ]:
# tinkering.ipynb or tinkering.py
"""
Initialize and test LLMService in notebook environment
"""

# Fix import paths for your current structure
import sys
import os

# Add the project root to Python path
project_root = os.path.dirname(os.path.abspath('.'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Option 1: Simple settings without pydantic
class SimpleSettings:
    def __init__(self):
        self.groq_api_key = os.getenv("GROQ_API_KEY", "")
        self.groq_model = "llama3-8b-8192"
        self.groq_temperature = 0.1
        self.groq_max_tokens = 1000

# Create settings instance
settings = SimpleSettings()

# Now import and initialize LLMService
import logging
from typing import List, Dict, Any
from groq import Groq

# Set up logging for notebook
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class LLMService:
    """Service for LLM operations using Groq"""
    
    def __init__(self, api_key: str = None):
        """Initialize LLM service with Groq client"""
        self.api_key = api_key or settings.groq_api_key
        if not self.api_key:
            raise ValueError("Groq API key is required. Set GROQ_API_KEY environment variable.")
        
        self.client = Groq(api_key=self.api_key)
        self.model = settings.groq_model
        self.temperature = settings.groq_temperature
        self.max_tokens = settings.groq_max_tokens
        
        logger.info(f"LLM service initialized with model: {self.model}")
    
    def generate_answer(self, question: str, context_chunks: List[Dict[str, Any]] = None) -> str:
        """Generate answer using Groq LLM with optional context"""
        try:
            if context_chunks:
                context = self._format_context(context_chunks)
                prompt = self._create_prompt(question, context)
            else:
                prompt = question
            
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {
                        "role": "system",
                        "content": "You are a helpful AI assistant. Answer questions clearly and concisely."
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                temperature=self.temperature,
                max_tokens=self.max_tokens,
                top_p=1,
                stream=False
            )
            
            answer = response.choices[0].message.content.strip()
            logger.info(f"Generated answer for question: {question[:50]}...")
            return answer
            
        except Exception as e:
            logger.error(f"Error generating answer: {str(e)}")
            return f"I apologize, but I encountered an error: {str(e)}"
    
    def _format_context(self, context_chunks: List[Dict[str, Any]]) -> str:
        """Format context chunks into a readable string"""
        if not context_chunks:
            return "No relevant context found."
        
        formatted_context = []
        for i, chunk in enumerate(context_chunks, 1):
            source = chunk.get('metadata', {}).get('source', 'Unknown')
            text = chunk.get('text', '')
            formatted_context.append(f"[Context {i} - Source: {source}]\n{text}")
        
        return "\n\n".join(formatted_context)
    
    def _create_prompt(self, question: str, context: str) -> str:
        """Create a well-structured prompt for the LLM"""
        return f"""Based on the following context, please answer the question.

Context:
{context}

Question: {question}

Answer:"""
    
    def simple_chat(self, message: str) -> str:
        """Simple chat without context - useful for testing"""
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {
                        "role": "user",
                        "content": message
                    }
                ],
                temperature=self.temperature,
                max_tokens=self.max_tokens
            )
            
            return response.choices[0].message.content.strip()
            
        except Exception as e:
            return f"Error: {str(e)}"

# ============================================================================
# NOTEBOOK TESTING SECTION
# ============================================================================

def test_llm_service():
    """Test the LLM service initialization and basic functionality"""
    
    print("=== Testing LLM Service ===\n")
    
    # Check if API key is set
    if not os.getenv("GROQ_API_KEY"):
        print("❌ GROQ_API_KEY not found in environment variables")
        print("Please set it with: os.environ['GROQ_API_KEY'] = 'your-api-key'")
        return None
    
    try:
        # Initialize LLM service
        print("🚀 Initializing LLM service...")
        llm_service = LLMService()
        print("✅ LLM service initialized successfully!")
        
        # Test simple chat
        print("\n📝 Testing simple chat...")
        test_message = "Hello! Can you tell me what 2+2 equals?"
        response = llm_service.simple_chat(test_message)
        print(f"Question: {test_message}")
        print(f"Answer: {response}")
        
        # Test with context
        print("\n📚 Testing with context...")
        sample_context = [
            {
                "text": "Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn from data.",
                "metadata": {"source": "ai_textbook.pdf"}
            }
        ]
        
        context_question = "What is machine learning?"
        context_answer = llm_service.generate_answer(context_question, sample_context)
        print(f"Question: {context_question}")
        print(f"Answer: {context_answer}")
        
        print("\n✅ All tests passed!")
        return llm_service
        
    except Exception as e:
        print(f"❌ Error initializing LLM service: {str(e)}")
        return None

# ============================================================================
# RUN TESTS
# ============================================================================

if __name__ == "__main__" or "ipykernel" in sys.modules:
    # Set your API key here if not in environment
    # os.environ["GROQ_API_KEY"] = "your-groq-api-key-here"
    
    # Run the test
    llm = test_llm_service()
    
    # If successful, you can now use llm for further testing
    if llm:
        print("\n🎉 LLM service ready for use!")
        print("You can now call:")
        print("- llm.simple_chat('your question')")
        print("- llm.generate_answer('question', context_chunks)")